In [7]:
import numpy as np
import pandas as pd

import torch

from gaussian_ring_grid_generator import GaussianRingSpaceTimeGrid
import encoder
import importlib
importlib.reload(encoder)

<module 'encoder' from '/home/prs392/codes/capstone/numerical-relativity-interpolation/dev/encoder_sr/encoder.py'>

In [8]:
model = encoder.SR(1, 16, 5)

In [9]:
model(torch.rand(5, 1, 32, 32, 32), torch.rand(50, 1, 4))

torch.Size([1, 2560])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
spaceTimeContext = GaussianRingSpaceTimeGrid(
    n_space_grid = 32,
    n_time_grid = 5,
    space_min_x = -5,
    space_max_x = 5,
    time_min_t = 0.5,
    time_max_t = 2.5
)

/home/prs392/codes/capstone/numerical-relativity-interpolation/dev/encoder_sr/gaussian_ring_grid_generator.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(GaussianRing(self.space_grid.cpu(), i, i/2)).unsqueeze(0) for i in self.time_axis]


In [5]:
test_grid = GaussianRingSpaceTimeGrid(
    n_space_grid = 32,
    n_time_grid = 4,
    space_min_x = -5,
    space_max_x = 5,
    time_min_t = 0.87,
    time_max_t = 2.37
)

In [6]:
def get_decimal_index(axis_values, value):
    closest_above = axis_values[axis_values >= value].min()
    closest_below = axis_values[axis_values <= value].max()
    index_below = np.where(axis_values == closest_below)[0][0]
    
    if closest_above == closest_below:
        return index_below
    
    return index_below + ((value - closest_below)/(closest_above - closest_below))

In [7]:
ts = []
xs = []
ys = []
zs = []

time_t = []
space_x = []
space_y = []
space_z = []

vals = []

for i in range(test_grid.values.shape[0]):
    t = get_decimal_index(spaceTimeContext.time_axis.cpu().numpy(), test_grid.time_axis[i].item())
    
    
    for j in range(test_grid.values.shape[2]):
        x = get_decimal_index(spaceTimeContext.space_axis.cpu().numpy(), test_grid.space_axis[j].item())
        
        
        for k in range(test_grid.values.shape[3]):
            y = get_decimal_index(spaceTimeContext.space_axis.cpu().numpy(), test_grid.space_axis[k].item())
            
            
            for l in range(test_grid.values.shape[4]):
                z = get_decimal_index(spaceTimeContext.space_axis.cpu().numpy(), test_grid.space_axis[l].item())
                
                ts.append(t)
                xs.append(x)
                ys.append(y)
                zs.append(z)
                
                time_t.append(test_grid.time_axis[i].item())
                space_x.append(test_grid.space_axis[j].item())
                space_y.append(test_grid.space_axis[k].item())
                space_z.append(test_grid.space_axis[l].item())
                
                vals.append(test_grid.values[i, 0, j, k, l].item())

In [8]:
inps = np.array([ts, xs, ys, zs]).T
outs = np.array(vals)
inps.shape, outs.shape

((131072, 4), (131072,))

In [9]:
input_tensor = torch.Tensor(inps).unsqueeze(1)
output_tensor = torch.Tensor(outs).unsqueeze(1)
input_tensor.shape, output_tensor.shape

(torch.Size([131072, 1, 4]), torch.Size([131072, 1]))

In [10]:
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(input_tensor,output_tensor) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=32768)

In [11]:
import torch.nn as nn

model = encoder.SR(1, 16, 5)

In [41]:
model.load_state_dict(torch.load('/scratch/prs392/capstone/checkpoints/encoder_sr/epoch=31.pt'))

<All keys matched successfully>

In [42]:
import tqdm

model.to(device)
model.eval()

outputs = []

running_loss = 0.0
for i, data in tqdm.tqdm(enumerate(test_dataloader, 0)):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward + backward + optimize
    output = model(spaceTimeContext.values, inputs)
#         print(inputs)
#         print(labels)
#     print(outputs.shape)
    
    outputs.append(output)


4it [01:11, 17.85s/it]


In [43]:
outputs[0].shape

torch.Size([32768, 1])

In [33]:
model_outputs = torch.cat(outputs, dim=0).squeeze(1).detach().cpu().numpy()

In [34]:
df = pd.DataFrame(list(zip(time_t, space_x, space_y, space_z, vals, list(model_outputs))), columns =['t', 'x', 'y', 'z', 'expected', 'predicted']) 

In [35]:
df[(np.isclose(df['t'], 0.87)) & (df['z'] == -5.000000)]

,t,x,y,z,expected,predicted
0,0.87,-5.0,-5.000000,-5.0,0.0,-0.170676
32,0.87,-5.0,-4.677419,-5.0,0.0,-0.166062
64,0.87,-5.0,-4.354839,-5.0,0.0,-0.161447
96,0.87,-5.0,-4.032258,-5.0,0.0,-0.158357
128,0.87,-5.0,-3.709677,-5.0,0.0,-0.157428
...,...,...,...,...,...,...
32608,0.87,5.0,3.709677,-5.0,0.0,-0.240910
32640,0.87,5.0,4.032258,-5.0,0.0,-0.247445
32672,0.87,5.0,4.354839,-5.0,0.0,-0.253989
32704,0.87,5.0,4.677419,-5.0,0.0,-0.260556


In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [37]:
subset = df[(np.isclose(df['t'], 0.87)) & (df['z'] == -5.000000)]
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=subset['x'], y=subset['y'], mode = 'markers', marker=dict(
        color=subset['expected']
    )),
    row=1, col=1
)

fig.show()

In [38]:
subset = df[(np.isclose(df['t'], 0.87)) & (df['z'] == -5.000000)]
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=subset['x'], y=subset['y'], mode = 'markers', marker=dict(
        color=subset['predicted']
    )),
    row=1, col=1
)

fig.show()

In [39]:
subset['expected'].max(),subset['expected'].min(),subset['expected'].std(),subset['expected'].mean()

(2.1840358290380163e-20, 0.0, 1.983562664314081e-21, 3.298939106343956e-22)

In [40]:
subset['predicted'].max(),subset['predicted'].min(),subset['predicted'].std(),subset['predicted'].mean()

(-0.14020097255706787,
 -0.26711082458496094,
 0.025788225811347692,
 -0.17099435938871466)